In [ ]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import os
import h5py
np.set_printoptions(suppress=True)

In [ ]:
total_path = r'Y:\zym\IS\activate_copy'
#total_path = r'Z:\zym\IS\test'
activate_path = total_path + '\\' + '6'
#activate_path = total_path + '\\' + 'test'
filelist_a = os.listdir(activate_path)
plx_a=[]
for f in filelist_a:
    if f.endswith('.plx'):
        plx_a.append(f[:-4])

In [ ]:
ptd_all = []
wave_data = []
unit_name = []
for idx,file_name in enumerate(plx_a):
    wave = pd.read_excel(os.path.join(activate_path, file_name+'waveform.xlsx'),engine='openpyxl')
    wave = wave[wave.peak_trough_distance != 0]
    wave_data.extend(wave.values[:,0:3].tolist())
    unit_name.extend((wave.values[:,0]).tolist())
unit_name = list(dict.fromkeys(unit_name))
unit_name.sort()
for u in unit_name:
    ptd = []
    for w in wave_data:
        if w[0] == u:
            ptd.append(w[2])
    ptd_all.append(np.mean(ptd))
ptd_avg = np.array(ptd_all)
narrow_idx = np.argwhere((ptd_avg<0.4)|(ptd_avg==0.4)).flatten()
narrow_idx = narrow_idx.tolist()
broad_idx = np.argwhere(ptd_avg>0.4).flatten()
broad_idx = broad_idx.tolist()

In [ ]:
unit_narrow = []
for i in narrow_idx:
    unit_narrow.append(unit_name[i])
unit_broad = []
for i in broad_idx:
    unit_broad.append(unit_name[i])

In [ ]:
unit_a = []
wave_a = pd.read_excel(os.path.join(activate_path, plx_a[0]+'waveform.xlsx'),engine='openpyxl')
wave_a = wave_a[wave_a.peak_trough_distance != 0]
unit_a.extend((wave_a.values[:,0]).tolist())
unit_a = list(dict.fromkeys(unit_a))
unit_a.sort()
timestamp_a = h5py.File(os.path.join(activate_path, plx_a[0]+'timestamp.mat'),'r')
#timestamp_b = h5py.File(os.path.join(baseline_path, plx_b[0]+'timestamp.mat'),'r')
kbd_a = timestamp_a['timestamp/kbd1'][:].flatten()
kbd_a

In [ ]:
FR_df = pd.DataFrame(columns=unit_a,index=['before_FR','after_FR','change','cell_type'])
narrow_df = pd.DataFrame(columns=unit_narrow,index=['before_FR','after_FR','change','cell_type'])
broad_df = pd.DataFrame(columns=unit_broad,index=['before_FR','after_FR','change','cell_type'])
marker_a = np.arange(20,111,22)+kbd_a

broad_inc = []
b_inc_idx = []
broad_dec = []
b_dec_idx = []
broad_noch = []
b_noch_idx = []
narrow_inc = []
n_inc_idx = []
narrow_dec = []
n_dec_idx = []
narrow_noch = []
n_noch_idx = []
#unit_list = list(wave_a.unit)
unit_name = []
gap = 10
for i in unit_a:
    name = 'timestamp/'+i
    unit_name.append(name)
    #fig,ax = plt.subplots(2,1,figsize=(6,4))
    spike_a = timestamp_a[name][:].flatten()
    spk_list=[]
    for m in marker_a:
        start = m - gap
        stop = m + gap
        spk = spike_a[(spike_a>start) & (spike_a<stop)]-start
        spkl = spk.tolist()
        spk_list.append(spkl)
    spk_array = np.asarray(spk_list)
    # ax[0].spines['top'].set_visible(False)
    # ax[0].spines['right'].set_visible(False)
    # ax[0].spines['bottom'].set_visible(False)
    # ax[0].spines['left'].set_visible(False)
    # ax[0].axes.xaxis.set_visible(False)
    # ax[0].eventplot(spk_array, 'horizontal',linelengths=1,linewidths=0.2,color='slategray')
    # ax[0].set_ylabel('#Trial',fontsize=14)
    # ax[0].set_ylim(-0.5,4.5)
    # ax[0].set_yticks(np.arange(0,4.1,1))
    # ax[0].set_xlim(0,2*gap)
    # ax[0].set_xticks(np.linspace(0,(2*gap/0.2)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    # ax[0].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    # ax[0].set_title(i)

    _hist = np.zeros((len(spk_array), len(np.arange(0, 2*gap+0.1, 0.2))-1))
    _hist2 = np.zeros((len(spk_array), len(np.arange(0, 2*gap+0.1, 1))-1))
    for row in range(len(spk_array)):
        spk_row = spk_array[row]
        distribution,_ = np.histogram(spk_row, np.arange(0, 2*gap+0.1, 0.2))
        distribution2,_ = np.histogram(spk_row, np.arange(0, 2*gap+0.1, 1))
        _hist[row] = distribution
        _hist2[row] = distribution2
    hist = np.sum(_hist, 0)
    #hist2 = np.sum(_hist2[:4,:],0)/4*5
    hist2 = np.sum(_hist2,0)
    hist2_front = hist2[0:int(len(hist2)/2)]
    hist2_back = hist2[int(len(hist2)/2):]
    #hist_mean = np.mean(hist, 0)
    #f1 = lagrange(np.arange(0, 2*gap, 0.2),hist)
    f1 = scipy.interpolate.interp1d(np.arange(0, gap, 1),hist2_front,kind='linear',fill_value="extrapolate")  #quadratic
    f2 = scipy.interpolate.interp1d(np.arange(gap, 2*gap, 1),hist2_back,kind='linear',fill_value="extrapolate")  #quadratic
    binfront = np.arange(0, gap, 0.1)
    binback = np.arange(gap, 2*gap, 0.1)
    hist_front = f1(binfront)
    hist_back = f2(binback)
    hist_new = np.concatenate((hist_front,hist_back))
    #hist2 = np.sum(_hist2, 0)
    # ax[1].bar((np.arange(0, 2*gap+0.1, 0.2)-0.1)[1:], hist, width = 0.2,color='slategray')
    # title = '%s_PSTH_5trial.jpg'% i
    # ax[1].spines['top'].set_visible(False)
    # ax[1].spines['right'].set_visible(False)
    # ax[1].spines['bottom'].set_linewidth('1')
    # ax[1].spines['left'].set_linewidth('1')
    # ax[1].set_ylabel('Counts',fontsize=14)
    # ax[1].set_xlim(0,2*gap)
    # ax[1].set_xticks(np.linspace(0,len(hist)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    # ax[1].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    # #plt.savefig(os.path.join(activate_path, title),dpi=400)
    # #plt.show()
    # plt.close()
    before_spk = hist[31:51]
    after_spk = hist[51:71]
    hist_evoke = hist_new-np.mean(hist2[0:50])
    #hist_zscore = stats.zscore(hist_evoke)
    #hist_zscore = hist_evoke
    hist_zscore = stats.zscore(hist_evoke)
    stim_mean = np.mean(after_spk)
    spon_mean = np.mean(before_spk)
    c95m = np.std(before_spk)*1.96/np.sqrt(len(before_spk))
    if stim_mean > (spon_mean+c95m): 
        c95 = 1
    elif stim_mean < (spon_mean-c95m): 
        c95 = 2
    else:
        c95 = 0
    print(c95)
    before_FR = np.sum(hist[30:50])/(20*0.2*5)
    after_FR = np.sum(hist[50:70])/(20*0.2*5)
    FR_df.loc['before_FR', i] = before_FR
    FR_df.loc['after_FR', i] = after_FR
    FR_df.loc['change', i] = c95
    if i in unit_broad:
        FR_df.loc['cell_type',i] = 'broad'
        FR_df.loc['cell_type',i] = 'broad'
        broad_df.loc['before_FR', i] = before_FR
        broad_df.loc['after_FR', i] = after_FR
        broad_df.loc['cell_type', i] = 'broad'
        broad_df.loc['change', i] = c95
        if c95 == 1:
            broad_inc.append(hist_zscore)
            b_inc_idx.append(i)
        elif c95 == 2:
            broad_dec.append(hist_zscore)
            b_dec_idx.append(i)
        else:
            broad_noch.append(hist_zscore)
            b_noch_idx.append(i)
    else:
        FR_df.loc['cell_type',i] = 'narrow'
        FR_df.loc['cell_type',i] = 'narrow'
        narrow_df.loc['before_FR', i] = before_FR
        narrow_df.loc['after_FR', i] = after_FR
        narrow_df.loc['cell_type', i] = 'narrow'
        narrow_df.loc['change', i] = c95
        if c95 == 1:
            narrow_inc.append(hist_zscore)
            n_inc_idx.append(i)
        elif c95 == 2:
            narrow_dec.append(hist_zscore)
            n_dec_idx.append(i)
        else:
            narrow_noch.append(hist_zscore)
            n_noch_idx.append(i)
FR_df.to_excel(os.path.join(activate_path,'FR.xlsx'))
narrow_df.to_excel(os.path.join(activate_path,'narrow_FR.xlsx'))
broad_df.to_excel(os.path.join(activate_path,'broad_FR.xlsx'))

broad_inc_df = pd.DataFrame(broad_inc,index=b_inc_idx)
broad_dec_df = pd.DataFrame(broad_dec,index=b_dec_idx)
broad_noch_df = pd.DataFrame(broad_noch,index=b_noch_idx)
narrow_inc_df = pd.DataFrame(narrow_inc,index=n_inc_idx)
narrow_dec_df = pd.DataFrame(narrow_dec,index=n_dec_idx)
narrow_noch_df = pd.DataFrame(narrow_noch,index=n_noch_idx)

broad_inc_df.to_excel(os.path.join(activate_path,'broad_inc_zscore.xlsx'))
broad_dec_df.to_excel(os.path.join(activate_path,'broad_dec_zscore.xlsx'))
broad_noch_df.to_excel(os.path.join(activate_path,'broad_noch_zscore.xlsx'))
narrow_inc_df.to_excel(os.path.join(activate_path,'narrow_inc_zscore.xlsx'))
narrow_dec_df.to_excel(os.path.join(activate_path,'narrow_dec_zscore.xlsx'))
narrow_noch_df.to_excel(os.path.join(activate_path,'narrow_noch_zscore.xlsx'))